In [ ]:
import coco_dataset
import pathlib
import show_utils
import matplotlib.pyplot as plt
from torchvision import transforms

In [ ]:
inp_size = (512,512)
targ_size = (inp_size[0] // 8, inp_size[1] // 8)
coco_dataset = coco_dataset.CocoKeypoints(
    root=str(pathlib.Path("../coco") / "images" / "train2017"),
    annFile=str(
        pathlib.Path("../coco")
        / "annotations"
        / "annotations"
        / "person_keypoints_train2017.json"
    ),
    transform=transforms.Resize(inp_size),
)
i=0

In [ ]:
i=0

In [ ]:
sample = coco_dataset[i]

tf_image, pafs, heatmaps, keypoints, image, target, mask_out = sample

print(image.size)


show_utils.show_coco(image, target, coco_dataset.coco, draw_bbox=False)

plt.figure()
plt.imshow(mask_out)

show_utils.show_annotated(tf_image, keypoints, size=targ_size)

plt.figure()
show_utils.show_heatmaps_combined(heatmaps)


# plt.figure()
# show_utils.show_pafs_combined(pafs)


# plt.figure()
# show_utils.show_pafs_quiver_combined(pafs, size=targ_size)

# plt.figure()
# show_utils.show_pafs(pafs)

# plt.figure()
# show_utils.show_heatmaps(heatmaps)

# show_utils.show_pafs_quiver(pafs, size=new_size)




print(i)
i += 1